In [1]:
import cv2
import mediapipe as mp
import numpy as np
import datetime
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
hold_time=8
rep_count=0
time_now=0
rep_counted=False


In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) 
    b = np.array(b) #middle angle
    c = np.array(c) 
    
    #calculating left_knee angle
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
        
    return angle 

In [4]:
cap = cv2.VideoCapture(r"C:\Users\Lithium\Downloads\KneeBendVideo.mp4")

#Setting mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if frame is not None: 
        
            # Colouring image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Making  detection
            results = pose.process(image)

            # Recoloring back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
        else:
            exit(1)
            
            

        # Extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Getting coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculating angle
            angle = calculate_angle(left_hip,left_knee,left_ankle)
            
            # Visualizing angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 230), 2, cv2.LINE_AA)
            
            #counting reps
            if angle>140:
                if hold_time<8:
                    image = cv2.putText(image, "Keep Your Knee bent", (20, 70),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                knee_bent=False
                
            if angle<140 and knee_bent==False:
                knee_bent=True
                rep_counted = False
                time_now=datetime.datetime.now()
                
            if knee_bent:
                hold_time = (datetime.datetime.now() - time_now).total_seconds()
                image = cv2.putText(image,"Time held:" + str( hold_time), (20, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            if hold_time>8 and rep_counted==False:
                rep_count += 1  
                rep_counted=True     
                               
        except:
            pass
        
        image = cv2.putText(image,"Rep count:" + str(rep_count), (20, 460),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,), 2, cv2.LINE_AA)
        
        
        # Rendering detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()